# Times of India Article Display Interface

In [1]:
from toidisplay import display_requested_articles, load_metadata, binary_categorize
import pandas as pd

In [2]:
# Add list-defining function here:

input_list = pd.read_csv('sample_input.csv', index_col=0).index.to_list()[:6]
output_csv = 'sample_output.csv'

## Display Articles

In [3]:
display_requested_articles(display_list=input_list, input_function=binary_categorize, chunk_size=3, append_mode=True, save_location=output_csv)

Progress saved.
All requested articles displayed!
